# Overall Model Training

Train and evaluate the model on the whole system(rectifier) and each capacitor(isolated or on the system)

In [23]:
import torch
import optuna
import pickle

import sys
sys.path.append('./data')
from datautil import create_loaders
from architectures import RUL_transition,RUL_estimation,AutoencoderMLP
from training_decoder import train_estimation,train_transition,train_autoencoder 

import os

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Data

In [25]:
#Data
data='Rectifier'#'-Rectifier_300Diode'
dataset_name='Rectifier'#'RectifierRUL'# 
exp_est='estimation_noisy0'#'RUL_estimation3'# 
exp_tra='transition3'#'prueba'#
dim=3
exp_aut=f'autoencoder_dim{dim}'
exp=f'overall{dim}'#'overall_noisy' #'overall3'


#### choose the component(if Rectifier)

In [26]:
exp_address=f'./Experiments/{data}/{dataset_name}_{exp}'
exp_address_tra=f'./Experiments/{data}/{dataset_name}_{exp_tra}'
exp_address_est=f'./Experiments/{data}/{dataset_name}_{exp_est}'
exp_address_aut=f'./Experiments/{data}/{dataset_name}_{exp_aut}'
#extended='' #'_ext'#

# Study

In [27]:
study_name_aut = exp_address_aut+'/estudio'
study_name_tra = exp_address_tra+'/estudio'
study_name_est = exp_address_est+'/estudio'
storage_name_tra = "sqlite:///{}.db".format(study_name_tra)
storage_name_est = "sqlite:///{}.db".format(study_name_est)
storage_name_aut = "sqlite:///{}.db".format(study_name_aut)
study_tra = optuna.load_study(study_name=study_name_tra, storage=storage_name_tra)
study_est = optuna.load_study(study_name=study_name_est, storage=storage_name_est)
study_aut = optuna.load_study(study_name=study_name_aut, storage=storage_name_aut)

In [28]:
study_aut.best_trial.params

{'layer3': 200,
 'layer2': 240,
 'layer1': 120,
 'emb_dim': 3,
 'n_epoch': 47,
 'lr': 0.0009128718305268314}

In [29]:
study_tra.best_trial.params

{'rul_head_dim': 64,
 'n_head': 11,
 'embed_dim/n_head': 6,
 'dim_feedforward': 128,
 'N_epoch': 25,
 'lr': 8.382056903708147e-05}

In [30]:
study_est.best_trial.params

{'rul_head_dim': 32,
 'n_head': 9,
 'embed_dim/n_head': 2,
 'dim_feedforward': 64,
 'N_epoch': 30,
 'noise_strength_ini': 7.475274613924334,
 'noise_strength_end': 5.605270196679893,
 'noise_bound_ini': 1.0496990752839725,
 'noise_bound_end': 0.5570464204713121,
 'lr': 0.0004231768025787699}

In [31]:
print(f'Autoencoder trial numb: {len(study_aut.trials)}')
print(f'Autoencoder best trial: {study_aut.best_trial.value}')
print(f'Transition trial numb: {len(study_tra.trials)}')
print(f'Transition best trial: {study_tra.best_trial.value}')
print(f'Estimation trial numb: {len(study_est.trials)}')
print(f'Estimation best trial: {study_est.best_trial.value}')


Autoencoder trial numb: 76
Autoencoder best trial: 0.4418268822639031
Transition trial numb: 100
Transition best trial: 0.02298082523611841
Estimation trial numb: 15
Estimation best trial: 10.858512707796121


### Create Loaders and train models using optimized hyperparameters

In [32]:
loss='MSE'
weighted=False


train_config_aut={'n_epoch':study_est.best_trial.params['N_epoch'],#500
            'lr':  study_est.best_trial.params['lr']}



train_config_tra={'n_epoch':study_tra.best_trial.params['N_epoch'],#500
            'lr':  study_tra.best_trial.params['lr'],#1e-4
            'a':10,#1 10
            'b':13, #20 13
            'alpha':1,
            'war':200,
            'Floss':loss,
            'weighted':weighted}

train_config_est={'n_epoch':study_est.best_trial.params['N_epoch'],#500
            'lr':  study_est.best_trial.params['lr'],#1e-4
            'a':10,#1 10
            'b':13, #20 13
            'alpha':1,
            'war':200,
            'Floss':loss,
            'weighted':weighted}



with open(f'./data/{data}/dataset.pkl', 'rb') as file:
    train_dataset = pickle.load(file)
    
with open(f'./data/{data}/dataset_Test.pkl', 'rb') as file:
    test_dataset = pickle.load(file)

window=30 
batch_size=64
noisy=True
train_loader, test_loader=create_loaders(train_dataset=train_dataset,test_dataset=test_dataset,window=window+1,batch_size=(batch_size,256),scored=noisy,stats=False)

#### Create models

In [33]:
config_aut=study_aut.best_trial.params
config_aut['n_features']=train_dataset.n_features
config_aut['window']=window
config_tra=study_tra.best_trial.params
config_tra['n_features']=train_dataset.n_features
config_tra['window']=window
config_est=study_est.best_trial.params
config_est['n_features']=train_dataset.n_features
config_est['window']=window

In [34]:
if noisy:
    noise_strength_ini=study_est.best_trial.params['noise_strength_ini']  
    noise_strength_end=study_est.best_trial.params['noise_strength_end' ]  
    noise_bound_ini=study_est.best_trial.params['noise_bound_ini']    
    noise_bound_end=study_est.best_trial.params['noise_bound_end']    
    noise=(noise_strength_ini,noise_bound_ini,noise_strength_end,noise_bound_end) 
else:
    noise=False

In [35]:
#study.best_trial.params['look_back']    

model_aut=AutoencoderMLP(dim=[config_aut['window'],config_aut['n_features']],
                layers=[config_aut['layer1'],config_aut['layer2'],config_aut['layer3']],
                emb_dim=config_aut['emb_dim']).to(device).double()




model_tra=RUL_transition(look_back=config_tra['window'],
                                        n_features=config_tra['n_features'],
                                        embed_dim=config_tra['embed_dim/n_head']*config_tra['n_head'],
                                        rul_head_dim=config_tra['rul_head_dim'],
                                        dim_feedforward=config_tra['dim_feedforward'],
                                        n_head=config_tra['n_head']
                                        ).to(device).double()

model_est=RUL_estimation(look_back=config_est['window'],
                                        n_features=config_est['n_features'],
                                        embed_dim=config_est['embed_dim/n_head']*config_est['n_head'],
                                        rul_head_dim=config_est['rul_head_dim'],
                                        dim_feedforward=config_est['dim_feedforward'],
                                        n_head=config_est['n_head']
                                        ).to(device).double()


from sklearn.svm import OneClassSVM
one_class_svm=OneClassSVM(nu=0.0001,kernel='rbf',gamma=0.005)

## Train estimation and transition models

In [36]:
# X_train=train_loader.dataset.x[:,:-1,:]
# X_train=X_train.reshape(X_train.shape[0],-1)

# one_class_svm_trained=one_class_svm.fit(X_train)

In [37]:
model_aut_trained=train_autoencoder(model_aut,train_loader,config_aut)
print('done')
#model_est_trained=train_estimation(model_est,train_loader,train_config_est,noise_coef=noise)
print('done')
#model_tra_trained=train_transition(model_tra,train_loader,train_config_tra)
print('done')


done
done
done


## Save dataset and learned model 

In [38]:
os.makedirs(exp_address, exist_ok=True)
torch.save(train_loader, f'{exp_address}/train_loader.pkl')
torch.save(test_loader, f'{exp_address}/test_loader.pkl')


# with open(f'{exp_address}/model_one_class_svm.pkl','wb') as f:
#     pickle.dump(one_class_svm,f)

    
weights_aut=model_aut_trained.state_dict()
#weights_tra=model_tra_trained.state_dict()
#weights_est=model_est_trained.state_dict()
torch.save(weights_aut, f'{exp_address}/model_aut.pt')
#torch.save(weights_tra, f'{exp_address}/model_tra.pt')
#torch.save(weights_est, f'{exp_address}/model_est.pt')
torch.save(config_aut, f'{exp_address}/config_aut.pkl')
torch.save(config_est, f'{exp_address}/config_est.pkl')
torch.save(config_tra, f'{exp_address}/config_tra.pkl')


## Load learned models and dataset

In [72]:
train_loader=torch.load(f'{exp_address}/train_loader.pkl')
test_loader=torch.load(f'{exp_address}/test_loader.pkl')


# with open(f'{exp_address}/model_one_class_svm.pkl', 'rb') as f:
#     one_class_svm = pickle.load(f)



weights_aut=torch.load(f'{exp_address}/model_aut.pt')
weights_tra=torch.load(f'{exp_address}/model_tra.pt')
weights_est=torch.load(f'{exp_address}/model_est.pt')

config_aut=torch.load(f'{exp_address}/config_aut.pkl')
config_est=torch.load(f'{exp_address}/config_est.pkl')
config_tra=torch.load(f'{exp_address}/config_tra.pkl')

In [73]:
f'{exp_address}/model_aut.pt'

'./Experiments/Rectifier/Rectifier_overall1/model_aut.pt'

In [47]:

X_train=train_loader.dataset.x[:,:-1,:]
#X_train=X_train.reshape(X_train.shape[0],-1)

In [48]:
X_train.shape

(45200, 30, 9)

In [52]:
model_aut_trained(X_train)

TypeError: flatten() takes from 0 to 1 positional arguments but 2 were given

In [296]:
from sklearn.svm import OneClassSVM
one_class_svm=OneClassSVM(nu=0.1,kernel='rbf',gamma='auto')
with open(f'{exp_address}/model_one_class_svm.pkl','wb') as f:
    pickle.dump(one_class_svm,f)
X_train=train_loader.dataset.x[:,:-1,:]
X_train=X_train.reshape(X_train.shape[0],-1)


one_class_svm_trained=one_class_svm.fit(X_train)

In [297]:
X_test=test_loader.dataset.x[:,:-1,:]
X_test=X_test.reshape(X_test.shape[0],-1)
prediction=one_class_svm.predict(X_test)

In [298]:
prediction.shape

(10950,)

In [299]:
sum(prediction)

7884

In [300]:
import numpy as np
X_false=torch.randn_like(torch.tensor(X_test))
X_false=X_false.numpy()

In [317]:
prediction=one_class_svm.predict(X_test+0.2*X_false)

In [318]:
sum(prediction)

6094